In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.signal as signal
import scipy.stats as stats
import plotly.io as pio
from IPython.display import display, HTML
from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.filters.cf_filter import cffilter
from statsmodels.tsa.stattools import adfuller

pio.renderers.default = "notebook_connected"

def show_plotly(fig, include_plotlyjs="inline"):
    """Helper do wyświetlania wykresów Plotly."""
    try:
        fig.show()
    except Exception:
        html = fig.to_html(full_html=False, include_plotlyjs='cdn')
        display(HTML(html))


Teraz tworzę funkcje pomocnicze, które pozwolą mi na łatwe wyznaczanie stóp zwrotu (prostą i logarytmiczną)
W dalszej części projektu skupię się na analizie **stóp logarytmicznych**.

In [2]:
def compute_returns(series: pd.Series) -> pd.DataFrame:
    returns = pd.DataFrame(index=series.index)
    returns["simple_frac"] = (series - series.shift(1)) / series.shift(1)
    returns["log_frac"] = np.log(series / series.shift(1))
    return returns

def compute_dft_complex(x):
    """
    Oblicza DFT (Dyskretną Transformatę Fouriera) przy użyciu algorytmu FFT.
    Zwraca współczynniki X_k (liczby zespolone).
    """
    return np.fft.fft(x)

def compute_naive_periodogram(x):
    """
    Oblicza periodogram naiwny na podstawie DFT.
    Zwraca wektor częstotliwości (f) i mocy (P).
    """
    N = len(x)
    X_k = compute_dft_complex(x)
    power = (1/N) * (np.abs(X_k)**2)
    freqs = np.fft.fftfreq(N, d=1)
    idx = np.where(freqs >= 0)
    return freqs[idx], power[idx]

In [3]:
data_btcusdt_frame_daily: pd.DataFrame = pd.read_csv(
    Path("./btcusd_day.csv"),
    usecols=["datetime", "open", "high", "low", "close"],
    parse_dates=["datetime"],
)
# Ustawiam odpowiednio indeks
data_btcusdt_frame_daily.set_index("datetime").sort_index().astype("float64")

# konwersja kolumny 'Data' na datetime, bo bez tego są błedne daty
data_btcusdt_frame_daily["datetime"] = pd.to_datetime(
    data_btcusdt_frame_daily["datetime"], errors="coerce"
)

# usuwam ewentualne błędne wiersze bez dat
data_btcusdt_frame_daily = data_btcusdt_frame_daily.dropna(subset=["datetime"])

# indeks i sortuje
data_btcusdt_frame_daily = (
    data_btcusdt_frame_daily.set_index("datetime").sort_index().astype("float64")
)

print(data_btcusdt_frame_daily.head(10))

               open     high      low    close
datetime                                      
2010-07-17  0.04951  0.04951  0.04951  0.04951
2010-07-18  0.08584  0.08584  0.08584  0.08584
2010-07-19  0.08080  0.08080  0.08080  0.08080
2010-07-20  0.07474  0.07474  0.07474  0.07474
2010-07-21  0.07921  0.07921  0.07921  0.07921
2010-07-22  0.05050  0.05050  0.05050  0.05050
2010-07-23  0.06262  0.06262  0.06262  0.06262
2010-07-24  0.05454  0.05454  0.05454  0.05454
2010-07-25  0.05050  0.05050  0.05050  0.05050
2010-07-26  0.05600  0.05600  0.05600  0.05600


In [4]:
strategy_equity_frame_daily: pd.DataFrame = pd.read_csv(
    Path("./equity_daily_normalized.csv"),
    usecols=["datetime", "equity"],
    parse_dates=["datetime"],
)
# strategy_equity_frame_daily: pd.DataFrame = pd.read_csv(
#     Path("./equity_daily_not_normalized.csv"),
#     usecols=["datetime", "equity"],
#     parse_dates=["datetime"],
# )

# Ustawiam odpowiednio indeks
strategy_equity_frame_daily.set_index("datetime").sort_index().astype("float64")

# konwersja kolumny 'Data' na datetime, bo bez tego są błedne daty
strategy_equity_frame_daily["datetime"] = pd.to_datetime(
    strategy_equity_frame_daily["datetime"], errors="coerce"
)

# usuwam ewentualne błędne wiersze bez dat
strategy_equity_frame_daily.dropna(subset=["datetime"])

# indeks i sortuje
strategy_equity_frame_daily = (
    strategy_equity_frame_daily.set_index("datetime").sort_index().astype("float64")
)

strategy_equity_frame_daily.head(10)

,equity
datetime,
2022-01-01 00:00:00+00:00,1.000000
2022-01-02 00:00:00+00:00,1.000000
2022-01-03 00:00:00+00:00,1.000000
2022-01-04 00:00:00+00:00,0.997370
2022-01-05 00:00:00+00:00,1.050057
2022-01-06 00:00:00+00:00,1.057094
2022-01-07 00:00:00+00:00,1.090937
2022-01-08 00:00:00+00:00,1.088179
2022-01-09 00:00:00+00:00,1.084254


In [5]:
# Wybór danych
# 1. Wybór danych (zgodnie z wyborem w GUI/kodu powyżej)
# raw_series = data_btcusdt_frame_daily['close'].values
raw_series = compute_returns(data_btcusdt_frame_daily['close'])["log_frac"]
# raw_series = strategy_equity_frame_daily['equity'].values

# 0 Przygotowanie danych i sprawdzenie stacjonarności

W materiałach $\lambda$ często jest ustawiona na 1600 dla danych kwartalnych. Natomiast dla dziennych często używa się 14400 lub 100000.

W literaturze (tzw. reguła Ravna-Uhliga), parametr $\lambda$ skaluje się z czwartą potęgą częstotliwości, co by nam dawało:

- kwartalnie: $1600$
- miesięczne: $1600 \times 3^4 = 129\ 600$ (często upraszczane do 144 000)
- dziennie (ok. 90 dni w kwartale): $1600 \times 90^4 \approx \mathbf{100\ 000\ 000} \ (10^8)$

Profesjonalnie dla danych dziennych powinniśmy więc użyć $\lambda=\mathbf{100\ 000\ 000} \ (10^8)$, aby trend był "gładki", a cykle wyraźne.
$10^8$ wygląda na ogromną wartość parametru, dlatego ja ustawiłem $\lambda=\mathbf{6 \ 400 \ 000}$ (standard dla wielu analiz finansowych).

Filtr CF
Pominać logarytmowwnie
dodać jednostkę czasu

In [6]:
# 1. Filtr Hodricka-Prescotta (HP)
print(f"Przetwarzanie serii HP: {len(raw_series)} punktów")

data_series_hp = raw_series.dropna()
hp_lambda = 6_400_000
cycle_hp, trend_hp = hpfilter(data_series_hp, lamb=hp_lambda)

def plot_hp_results(raw, trend, cycle):
    fig = make_subplots(rows=3, cols=1, 
                         subplot_titles=("1. Surowe log-zwroty (Dane wejściowe)", 
                                         "2. Wyizolowany trend (Low-pass component)", 
                                         "3. Składowa cykliczna (Stacjonarna - wysokoczęstotliwościowa)"))
    
    fig.add_trace(go.Scatter(y=raw, name="Raw"), row=1, col=1)
    fig.add_trace(go.Scatter(y=trend, name="Trend"), row=2, col=1)
    fig.add_trace(go.Scatter(y=cycle, name="Cycle"), row=3, col=1)
    
    fig.update_layout(title_text="Filtr Hodricka-Prescotta (HP)", height=900, showlegend=False)
    show_plotly(fig)

plot_hp_results(data_series_hp, trend_hp, cycle_hp)

Przetwarzanie serii HP: 5607 punktów


### Dlaczego wykresy wyglądają tak, a nie inaczej?

#### 1. HP: Podobieństwo "Raw" vs "Cycle"
Filtr Hodricka-Prescotta jest filtrem typu górnoprzepustowego (high-pass). Ponieważ analizujemy teraz **log-zwroty** (które same w sobie są już wynikiem filtrowania cen i są stacjonarne), filtr HP nie ma w nich "czego wycinać" poza bardzo długoterminowym trendem. Jeśli $\lambda$ jest wysokie (np. 6.4M), trend jest prawie płaską linią przy zerze. Dlatego `cykl = dane - trend` wygląda prawie identycznie jak dane wejściowe. To dowodzi, że zwroty są już "wysokoczęstotliwościowe".

#### 2. CF: Problem stacjonarności i brzegów
- **Efekty brzegowe**: Na samym początku wykresu CF widać gwałtowny spadek (do ok. -0.2). Jest to typowy artefakt filtrów pasmowoprzepustowych przy krawędziach danych (tzw. end effects). Filtr potrzebuje "okna" danych, by poprawnie działać.
- **Parametry [500, 1500]**: Przy tak długich cyklach (halving to ok. 1460 dni), ADF może zgłaszać niestacjonarność, jeśli okno jest zbyt szerokie, bo sygnał staje się zbyt gładki i "leniwy". Zmieniliśmy zakres na 500-1500 dni, co daje p-value < 0.05, czyniąc szereg stacjonarnym w sensie statystycznym, przy jednoczesnym zachowaniu kluczowych cykli.

In [17]:
# 2. Filtr Christiano-Fitzgeralda (CF)
# Filtr pasmowoprzepustowy - isolujemy cykle [100, 1500] dni
print(f"Przetwarzanie serii CF: {len(raw_series)} punktów")

data_series_cf = raw_series.dropna()
# low=100, high=1500, drift=False (dla zwrotów lepiej bez driftu)
cycle_cf, trend_cf = cffilter(data_series_cf, low=100, high=1500, drift=False)

def plot_cf_results(raw, trend, cycle):
    fig = make_subplots(rows=3, cols=1, 
                         subplot_titles=("1. Surowe log-zwroty (Dane wejściowe)", 
                                         "2. Wyizolowany trend (Slow-moving components)", 
                                         "3. Składowa cykliczna (Cykle [100, 1500] dni)"))
    
    fig.add_trace(go.Scatter(y=raw, name="Raw"), row=1, col=1)
    fig.add_trace(go.Scatter(y=trend, name="Trend"), row=2, col=1)
    fig.add_trace(go.Scatter(y=cycle, name="Cycle (Band-passed)"), row=3, col=1)
    
    fig.update_layout(title_text="Filtr Christiano-Fitzgeralda (CF)", height=1000, showlegend=False)
    show_plotly(fig)

plot_cf_results(data_series_cf, trend_cf, cycle_cf)

Przetwarzanie serii CF: 5607 punktów


### Sprawdzenie stacjonarności danych

Zanim przejdziemy do punktu 1. projektu, sprawdzimy czy dane po filtracji są stacjonarne.

In [18]:
# Weryfikacja stacjonarności (Test ADF) ---
def check_stationarity(timeseries, name: str):
    print(f"-" * 60)
    print(f"Test stacjonarności (ADF) dla: {name}")
    
    # Usuwamy NaN jeśli są
    ts = timeseries.dropna() if hasattr(timeseries, 'dropna') else timeseries

    result = adfuller(ts, autolag="AIC")
    p_value = result[1]
    print(f"Statystyka ADF: {result[0]:.4f}")
    print(f"p-value: {p_value:.10f}")

    if p_value <= 0.05:
        print("p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.")
    else:
        print("p-value > 0.05. Nie ma podstaw do odrzucenia H0. Szereg jest NIESTACJONARNY.")

check_stationarity(raw_series, "Log-zwroty (Raw)")
check_stationarity(cycle_hp, "Składowa cykliczna HP")
check_stationarity(cycle_cf, "Składowa cykliczna CF")

------------------------------------------------------------
Test stacjonarności (ADF) dla: Log-zwroty (Raw)
Statystyka ADF: -12.8864
p-value: 0.0000000000
p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.
------------------------------------------------------------
Test stacjonarności (ADF) dla: Składowa cykliczna HP
Statystyka ADF: -27.4483
p-value: 0.0000000000
p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.
------------------------------------------------------------
Test stacjonarności (ADF) dla: Składowa cykliczna CF
Statystyka ADF: -3.6323
p-value: 0.0051719292
p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.


Zastosowanie testu Augmented Dickey-Fuller (ADF) potwierdziło stacjonarność składowej cyklicznej uzyskanej po filtracji HP ($p-value \approx 0.0$). Wysoka wartość parametru lambda ($6.4 \times 10^6$) pozwoliła na precyzyjne odizolowanie długookresowego trendu przy jednoczesnym zachowaniu istotnych informacji o cyklach rynkowych. Periodogram naiwny wykazuje silną koncentrację mocy w zakresie niskich częstotliwości ($f < 0.05$), co sugeruje dominację cykli trwających powyżej 20 dni. Ze względu na dużą wariancję estymatora naiwnego (charakterystyczny 'las szpilek'), w kolejnym kroku konieczne jest zastosowanie wygładzania spektralnego.

In [19]:
# Obliczamy periodogramy dla wszystkich składowych
f_ret, p_ret = compute_naive_periodogram(raw_series.dropna())
f_hp, p_hp = compute_naive_periodogram(cycle_hp)
f_cf, p_cf = compute_naive_periodogram(cycle_cf)

# Tworzymy wykres porównawczy
fig = go.Figure()

fig.add_trace(go.Scatter(x=f_ret[1:], y=p_ret[1:], name="Log-zwroty (Raw)", line=dict(color='gray', width=1)))
fig.add_trace(go.Scatter(x=f_hp[1:], y=p_hp[1:], name="Cykl HP", line=dict(color='blue', width=1.5)))
fig.add_trace(go.Scatter(x=f_cf[1:], y=p_cf[1:], name="Cykl CF", line=dict(color='orange', width=2)))

fig.update_layout(
    title="Porównanie periodogramów (Zoom na niskie częstotliwości)",
    xaxis_title="Częstotliwość (f)",
    yaxis_title="Moc (Power)",
    yaxis_type="log",
    xaxis_range=[0, 0.05], # ZOOM: skupiamy się na cyklach > 20 dni (1/0.05)
    hovermode="x unified"
)
show_plotly(fig)

### Wyjaśnienie "dziwnego" wyglądu filtra CF:

1. **Dlaczego periodogram CF to gładka linia?**
To nie błąd, to **dowód działania filtra**. Filtr CF jest filtrem pasmowoprzepustowym (band-pass). Ustawiliśmy go na zakres 100-1500 dni ($f \in [0.00067, 0.01]$). 
- To, co widzisz jako "wahania" na początku wykresu (lewa strona), to właśnie Twoje wyizolowane cykle.
- Wszystko na prawo od $f = 0.01$ (czyli 98% wykresu) zostało przez filtr **wycięte do zera**. Gładka linia schodząca w dół na skali logarytmicznej to po prostu charakterystyka wygaszania filtra. Na skali liniowej ta linia byłaby płaska przy samym zerze.

2. **Dlaczego 100-1500 dni jest niestacjonarne?**
Przy `low=100`, filtr dopuszcza zbyt dużo szybkich zmian, które przy silnym halvingu i specyfice Bitcoina mogą dawać szereg, który ADF uznaje za niestacjonarny (pamiętajmy, że ADF szuka pierwiastka jednostkowego). Ustawienie `low=500` (lub więcej) mocniej wygładza szereg, czyniąc go statystycznie stacjonarnym. W przypadku finansów, analiza cykli 100-dniowych jest jednak bardzo ciekawa, więc zostawiamy ją, akceptując fakt, że szereg jest "na granicy" stacjonarności.

3. **Drift=False**: Wyłączenie driftu sprawia, że składowa trendu nie próbuje na siłę dopasować się do wolnych zmian poziomu zwrotów, co daje stabilniejszą bazę cykliczną.

## Eksperyment: Podzielność liczby obserwacji N przez naturalny okres T

W tym punkcie badamy zjawisko **przecieku widma (spectral leakage)**. Teoria sugeruje, że jeśli liczba obserwacji $N$ nie jest wielokrotnością naturalnego okresu $T$ występującego w danych, energia sygnału "rozlewa się" na sąsiednie częstotliwości.

W danych finansowych za naturalne okresy możemy przyjąć:
* $T = 7$ (okres tygodniowy),
* $T = 365$ (okres roczny).

In [20]:
# Konfiguracja eksperymentu
natural_period = 365  # Cykl roczny
max_p = None        # None = bierze maksymalną możliwą liczbę okresów z całych danych

def run_divisibility_experiment(data, T, max_periods=None):
    total_N = len(data)
    
    # Największa możliwa liczba okresów w dostępnych danych
    possible_periods = total_N // T
    
    # Ustalamy ile okresów bierzemy
    if max_periods is None or max_periods > possible_periods:
        num_periods = possible_periods
    else:
        num_periods = max_periods
        
    N_div = num_periods * T
    
    # N2: Liczba niepodzielna przez T (staramy się być blisko N_div)
    # Wybieramy N_indiv tak, aby "psuło" pełny okres, czyli by było niepodzielne
    if N_div + (T // 2) <= total_N:
        N_indiv = N_div + (T // 2)
    else:
        # Jeśli nie ma miejsca na offset, bierzemy total_N (lub total_N - 1 jeśli total_N jest podzielne)
        N_indiv = total_N if total_N % T != 0 else total_N - 1

    # Finalna korekta niepodzielności (na wypadek gdyby T//2 też dawało podzielność)
    if N_indiv % T == 0:
        N_indiv -= 1

    print(f"Eksperyment dla T={T}:")
    print(f"Próbka podzielna (N_div): {N_div} ({N_div // T} pełnych okresów)")
    print(f"Próbka niepodzielna (N_indiv): {N_indiv}")

    # Wycinamy fragmenty danych (od początku serii)
    series_div = data[:N_div]
    series_indiv = data[:N_indiv]

    # Obliczamy periodogramy
    f_div, p_div = compute_naive_periodogram(series_div)
    f_indiv, p_indiv = compute_naive_periodogram(series_indiv)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=f_div[1:], y=p_div[1:],
        name=f"N={N_div} (Podzielne przez {T})",
        line=dict(color='green', width=1.2)
    ))

    fig.add_trace(go.Scatter(
        x=f_indiv[1:], y=p_indiv[1:],
        name=f"N={N_indiv} (Niepodzielne przez {T})",
        line=dict(color='red', width=1, dash='dot')
    ))

    fig.update_layout(
        title=f"Eksperyment: Wpływ podzielności N przez T={T} na periodogram",
        xaxis_title="Częstotliwość (f)",
        yaxis_title="Moc (Power)",
        yaxis_type="log",
        hovermode="x unified"
    )
    
    show_plotly(fig)

run_divisibility_experiment(cycle_cf, natural_period, max_periods=max_p)

Eksperyment dla T=365:
Próbka podzielna (N_div): 5475 (15 pełnych okresów)
Próbka niepodzielna (N_indiv): 5606


### Komentarz do eksperymentu i wnioski:

Przeprowadzony eksperyment porównujący periodogram dla liczby obserwacji $N$ podzielnej przez $T=7$ (lub $T=365$) oraz niepodzielnej, prowadzi do następujących wniosków:

1. **Brak wyraźnych różnic wizualnych**: Na powyższym wykresie obie linie (zielona i czerwona) niemal całkowicie się pokrywają (w oddali bez przybliżenia). Efekt "przecieku widma" (spectral leakage), który w teorii powinien powodować rozmycie pików przy $N$ niepodzielnym, jest w tym przypadku niewidoczny dla oka.
2. **Przyczyny braku widoczności efektu**:
   * **Wysoka wariancja periodogramu naiwnego**: Dominującą cechą periodogramu naiwnego jest to, że jest on estymatorem niezgodnym, jego wykres to ogromna ilość pojedynczych pików. Ta naturalna zmienność (szum) całkowicie przykrywa drobne, deterministyczne zniekształcenia wynikające z braku podzielności $N$ przez $T$.
   * **Niedeterministyczny charakter danych**: Bitcoin to realny szereg ekonomiczny, a nie czysta sinusoida. Cykle w takich danych nie są idealnie stałe w czasie, co powoduje naturalne "rozmycie" energii w widmie, niezależnie od doboru $N$.
   * **Duża liczba obserwacji ($N > 5000$)**: Przy tak dużej próbie, rozdzielczość częstotliwościowa jest bardzo wysoka. Błędy wynikające z niedopasowania okna do okresu są rozłożone na bardzo wąskie pasma, co czyni je nieistotnymi w skali całego widma.

## Podsumowanie Punktu 1

Analiza samym periodogramem naiwnym ( DFT ) jest niewystarczająca do precyzyjnej identyfikacji cykli. Duża wariancja estymatora maskuje zarówno subtelne zjawiska teoretyczne (jak leakage), jak i faktyczne cykle koniunkturalne. Stanowi to bezpośrednie uzasadnienie dla przejścia do **Fazy 2: Porównanie metod**, która ma na celu redukcję tej wariancji u wygładzenia periodogramu.

# 2. Porównanie metod

W tej części projektu przechodzimy do bardziej zaawansowanych metod estymacji gęstości widmowej mocy. Naszym celem jest redukcja wariancji periodogramu naiwnego, który jak pokazaliśmy wcześniej jest estymatorem niezgodnym.

## Periodogram wprost z definicji z przedziałem ufności 95%

Zgodnie z teorią, dla dużych $N$, statystyka $2I(f)/S(f)$ ma rozkład chi-kwadrat z 2 stopniami swobody ($\chi^2_2$). Pozwala to na wyznaczenie przedziału ufności dla prawdziwej gęstości widmowej $S(f)$ na podstawie wyznaczonego periodogramu $I(f)$:

$$ \frac{2 I(f)}{\chi^2_{1-\alpha/2}(2)} \leq S(f) \leq \frac{2 I(f)}{\chi^2_{\alpha/2}(2)} $$

Dla poziomu ufności 95% ($\alpha = 0.05$), wyznaczamy kwantyle rozkładu $\chi^2$ dla $0.975$ oraz $0.025$.


In [21]:
from scipy.stats import chi2

def plot_periodogram_with_ci(freqs, power, alpha=0.05, title="Periodogram z przedziałem ufności 95%"):
    # Kwantyle rozkładu chi-kwadrat z 2 stopniami swobody
    chi_low = chi2.ppf(alpha / 2, 2)
    chi_high = chi2.ppf(1 - alpha / 2, 2)
    
    # Wyznaczamy granice przedziału ufności
    lower_bound = (2 * power) / chi_high
    upper_bound = (2 * power) / chi_low
    
    fig = go.Figure()
    
    # Periodogram
    fig.add_trace(go.Scatter(
        x=freqs, y=power,
        name="Periodogram naiwny",
        line=dict(color='blue', width=1.5)
    ))
    
    # Górna granica
    fig.add_trace(go.Scatter(
        x=freqs, y=upper_bound,
        name=f"Górna granica CI ({100*(1-alpha):.0f}%)",
        line=dict(color='rgba(255, 0, 0, 0.3)', width=1, dash='dot'),
        showlegend=True
    ))
    
    # Dolna granica
    fig.add_trace(go.Scatter(
        x=freqs, y=lower_bound,
        name=f"Dolna granica CI ({100*(1-alpha):.0f}%)",
        line=dict(color='rgba(255, 0, 0, 0.3)', width=1, dash='dot'),
        fill='tonexty', # Wypełnienie między granicami
        fillcolor='rgba(255, 0, 0, 0.1)',
        showlegend=True
    ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Częstotliwość (f)",
        yaxis_title="Moc (Power)",
        yaxis_type="log",
        hovermode="x unified",
        height=600
    )
    
    show_plotly(fig)

# Rysujemy periodogram z 95% przedziałem ufności
# Pomijamy f=0 dla lepszej czytelności (skala log)
plot_periodogram_with_ci(f_cyc[1:], p_cyc[1:], alpha=0.05, title="Periodogram naiwny z 95% przedziałem ufności")

NameError: name 'f_cyc' is not defined

### Wnioski i Interpretacja:

1. **Szerokość przedziału ufności**: Zastosowanie skali logarytmicznej na osi Y pozwala zauważyć, że przedział ufności ma stałą szerokość w sensie proporcjonalnym. Oznacza to, że błąd względny estymacji jest taki sam dla każdej częstotliwości. Bandy dolna i górna są tak samo oddalone od wartości Y.
2. **Niezgodność estymatora**: Przedział ufności jest bardzo szeroki. Fakt, że nie zwęża się on wraz ze wzrostem $N$ (co wykazaliśmy teoretycznie), potwierdza, że periodogram naiwny nie zbiega do prawdziwej wartości gęstości widmowej.
3. **Piki istotne statystycznie**: Dopiero pik, który wyraźnie "wystaje" ponad nasze tło i szum oraz którego przedział ufności nie pokrywa się z szerokim pasmem szumu, może być uznany za potencjalny cykl (wyróżniający się pik na przestrzeni danych). W przypadku periodogramu naiwnego większość szpilek jest wynikiem losowości, a nie faktycznych cykli, co widać po tym, jak gęsto szpile przeskakują między górną a dolną granicą oraz jak bardzo są podobne.

## Wygładzanie spektralne przy użyciu okna Daniella

Jednym z najprostszych sposobów na otrzymanie zgodnego estymatora gęstości widmowej jest wygładzanie periodogramu naiwnego w dziedzinie częstotliwości. Metoda ta polega na uśrednianiu wartości periodogramu w otoczeniu danej częstotliwości (średnia ruchoma).

Estymator z oknem Daniella o szerokości $L = 2m + 1$ dany jest wzorem:
$$ \hat{S}(f_k) = \frac{1}{2m+1} \sum_{j=-m}^{m} I(f_{k+j}) $$

Zwiększenie szerokości okna ($m$) powoduje redukcję wariancji estymatora kosztem zwiększenia obciążenia (rozmycie pików). Wygładzony periodogram staje się estymatorem zgodnym, gdy $m \to \infty$ wraz z $N \to \infty$, ale wolniej niż $N$.

In [ ]:
def apply_daniell_smoothing(power, m):
    """
    Wygładzanie oknem Daniella (prosta średnia ruchoma) dla periodogramu.
    m: połowa szerokości okna (okno ma rozmiar 2m+1).
    """
    window_size = 2 * m + 1
    window = np.ones(window_size) / window_size
    
    # Używamy np.convolve. Dla brzegów stosujemy tryb 'same'
    smoothed_power = np.convolve(power, window, mode='same')
    return smoothed_power

# Parametry wygładzania
m_values = [1, 3, 7, 15]

fig = go.Figure()

# Periodogram naiwny dla porównania
fig.add_trace(go.Scatter(
    x=f_cyc[1:], y=p_cyc[1:],
    name="Naiwny I(f)",
    line=dict(color='rgba(0,0,255,0.2)', width=1)
))

colors = ['red', 'green', 'orange', 'blue']
for m, color in zip(m_values, colors):
    p_smoothed = apply_daniell_smoothing(p_cyc, m)
    fig.add_trace(go.Scatter(
        x=f_cyc[1:], y=p_smoothed[1:],
        name=f"Daniell (m={m}, L={2*m+1})",
        line=dict(color=color, width=2)
    ))

fig.update_layout(
    title="Wygładzanie periodogramu oknem Daniella",
    xaxis_title="Częstotliwość (f)",
    yaxis_title="Moc (Power)",
    yaxis_type="log",
    hovermode="x unified",
    height=600
)

show_plotly(fig)

### Wnioski i Interpretacja:

1. **Redukcja wariancji**: Wyraźnie widać, że wraz ze wzrostem szerokości okna ($m$), wykres staje się gładszy. Krótkookresowe oscylacje (szum) zostają wytłumione.
2. **Rozdzielczość vs Stabilność**: Dla małych $m$ (np. $m=1$) zachowujemy piki, ale wariancja jest nadal duża. Dla dużych $m$ (np. $m=7$) estymator jest bardzo stabilny, ale możemy stracić informację o blisko położonych cyklach (piki zlewają się w jeden szeroki garb).
3. **Identyfikacja cykli**: Wygładzony wykres pozwala łatwiej dostrzec dominujące składowe, które w wersji naiwnej mogły być przesłonięte przez sąsiednie szpilki o wysokiej amplitudzie, będące wynikiem losowości. Dzięki temu widzimy potencjalny cykl na częstotliwości **0.144**.

## Metoda Welcha (uśrednianie periodogramów)

Metoda Welcha jest udoskonaleniem metody Bartletta.
Polega na:
1. Podziale sygnału na segmenty (często nakładające się).
2. Nałożeniu okna (np. Hamminga, Hanna) na każdy segment w celu redukcji wycieku spektralnego.
3. Obliczeniu periodogramu dla każdego segmentu.
4. Uśrednieniu otrzymanych periodogramów.

Uśrednianie powoduje redukcję wariancji proporcjonalnie do liczby segmentów ($K$). Kosztem jest utrata rozdzielczości częstotliwościowej (ponieważ segmenty są krótsze niż cały sygnał $N$).

In [ ]:
from scipy.signal import welch

def compute_welch_periodogram(x, nperseg=256, noverlap=None):
    """
    Oblicza estymator gęstości widmowej metodą Welcha.
    """
    # nperseg: długość każdego segmentu
    # noverlap: liczba punktów nakładania się segmentów (domyślnie nperseg // 2)
    f, p = welch(x, fs=1.0, window='hamming', nperseg=nperseg, 
                 noverlap=noverlap, scaling='density', detrend=False)
    return f, p

# Testujemy różne długości segmentów
nperseg_values = [128, 256, 384, 512]

fig = go.Figure()

# Periodogram naiwny dla tła
fig.add_trace(go.Scatter(
    x=f_cyc[1:], y=p_cyc[1:],
    name="Naiwny I(f)",
    line=dict(color='rgba(0,0,255,0.15)', width=1)
))

colors = ['red', 'green', 'orange', 'blue']
for nps, color in zip(nperseg_values, colors):
    fw, pw = compute_welch_periodogram(cycle, nperseg=nps) # Używamy cycle
    fig.add_trace(go.Scatter(
        x=fw[1:], y=pw[1:],
        name=f"Welch (nperseg={nps})",
        line=dict(color=color, width=2)
    ))

fig.update_layout(
    title="Estymacja metodą Welcha dla różnych długości segmentów",
    xaxis_title="Częstotliwość (f)",
    yaxis_title="Moc (Power)",
    yaxis_type="log",
    hovermode="x unified",
    height=600
)

show_plotly(fig)

### Wnioski i Interpretacja:

1. **Gładkość estymatora**: Metoda Welcha daje najgładsze wyniki spośród dotychczasowych metod. Wynika to z bezpośredniego uśredniania niezależnych (w dużym stopniu) estymatów z segmentów.
2. **Wpływ `nperseg`**: 
    - Krótsze segmenty (`nperseg=128`) dają bardzo gładki wykres (mała wariancja), ale piki są szerokie i mało precyzyjne.
    - Dłuższe segmenty (`nperseg=512`) pozwalają na lepsze rozróżnienie bliskich częstotliwości, ale wykres jest bardziej "poszarpany".
3. **Stabilność**: Metoda Welcha jest uważana za standard w praktycznej analizie widmowej ze względu na doskonały balans między redukcją szumu a zachowaniem cech sygnału.

## Porównanie metod i analiza dominujących cykli

W tym punkcie zestawiamy wyniki z różnych metod estymacji. **Ważna uwaga:** Do analizy szczytów używamy składowej cyklicznej (`cycle`), ponieważ surowy szereg (lub nawet jego logarytm) jest zdominowany przez trend. W takim przypadku wykres gęstości widmowej opada niemal monotonicznie (tzw. szum czerwony), co uniemożliwia znalezienie lokalnych maksimów. Filtracja HP pozwala nam skupić się na właściwych oscylacjach.

In [ ]:
# Wybieramy najlepsze parametry dla każdej metody do ostatecznego porównania
m_best = 15
nps_best = 384

f_daniell = f_cyc
p_daniell = apply_daniell_smoothing(p_cyc, m_best)
f_welch, p_welch = compute_welch_periodogram(cycle, nperseg=nps_best)

# --- Analiza Szczytów (Peak Analysis) ---
from scipy.signal import find_peaks
import pandas as pd
import numpy as np

# Ustawienia (zgodnie z Twoją modyfikacją)
threshold = np.mean(p_welch) * 0.00001
prominence_val = np.max(p_welch) * 0.0003

peaks_idx, _ = find_peaks(p_welch, height=threshold, prominence=prominence_val)
peak_freqs = f_welch[peaks_idx]
peak_powers = p_welch[peaks_idx]

results = []
for f, p in zip(peak_freqs, peak_powers):
    # Pomijamy artefakty o bardzo niskiej f (okres > 200 dni) - m.in. ten pik 256d
    if f > 0.005:
        period = 1/f 
        results.append({"Częstotliwość": f, "Moc": p, "Okres (dni)": period})

# --- Wizualizacja Porównawcza ze znacznikami cykli ---
fig = go.Figure()

fig.add_trace(go.Scatter(x=f_cyc[1:], y=p_cyc[1:], name="Naiwny I(f)", 
                         line=dict(color='gray', width=1, dash='dot')))
fig.add_trace(go.Scatter(x=f_daniell[1:], y=p_daniell[1:], name=f"Daniell (m={m_best})", 
                         line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=f_welch[1:], y=p_welch[1:], name=f"Welch (nperseg={nps_best})", 
                         line=dict(color='red', width=2)))

# Dodajemy pionowe linie i etykiety dla wykrytych cykli
for res in results:
    f_peak = res["Częstotliwość"]
    t_peak = res["Okres (dni)"]
    
    # Linia pionowa
    fig.add_vline(x=f_peak, line_width=1, line_dash="dash", line_color="green",
                  opacity=0.6)
    
    # Adnotacja tekstowa na górze wykresu
    fig.add_annotation(x=f_peak, y=1.0, yref="paper",
                       text=f"{t_peak:.1f}d", showarrow=False,
                       textangle=-90, xanchor="left",
                       font=dict(size=10, color="green"))

fig.update_layout(
    title="Porównanie metod i identyfikacja dominujących cykli",
    xaxis_title="Częstotliwość (f)",
    yaxis_title="Moc (Power)",
    yaxis_type="log",
    hovermode="x unified",
    height=700
)

show_plotly(fig)

print("Zidentyfikowane dominujące cykle (metoda Welcha):")
if len(results) > 0:
    df_peaks = pd.DataFrame(results).sort_values("Moc", ascending=False)
    display(df_peaks)
else:
    print("Nie znaleziono wyraźnych szczytów.")

### Wnioski końcowe Faz 2:

1. **Dominujący cykl**: Analiza szczytów wskazuje na istnienie silnych składowych okresowych. Przykładowo, okres rzędu ~... dni (należy odczytać z tabeli powyżej) odpowiada ...
2. **Praktyczna użyteczność**: Metody wygładzania (Daniell, Welch) pozwoliły na odsianie szumu i skupienie się na faktycznych procesach cyklicznych w cenie Bitcoina. Wersja naiwna była zbyt nieczytelna do pewnej interpretacji.
3. **Stabilność wyników**: Szczyty zidentyfikowane metodą Welcha są zazwyczaj bardziej wiarygodne, ponieważ wynikają z uśredniania po segmentach, co redukuje prawdopodobieństwo uznania przypadkowej korelacji za cykl.

# Punkt 3: Analiza Danych Niekompletnych

W rzeczywistych zastosowaniach często spotykamy się z brakiem danych. W tej części symulujemy utratę 30% obserwacji i sprawdzamy, jak radzą sobie z tym standardowy periodogram oraz periodogram Lomba-Scargle'a.

In [ ]:
import numpy as np
import scipy.signal as signal
import plotly.graph_objects as go

# 1. Symulacja brakujących danych (30%)
np.random.seed(42)
# Używamy cyklu (tak jak w pkt 2), aby widzieć piki
y = cycle.values if hasattr(cycle, 'values') else cycle
N_total = len(y)
missing_rate = 0.3
mask = np.random.rand(N_total) > missing_rate

t_full = np.arange(N_total)
t_missing = t_full[mask]
data_missing = y[mask]

print(f"Oryginalna liczba punktów: {N_total}")
print(f"Liczba punktów po usunięciu 30%: {len(data_missing)}")

# 2. Periodogram Naiwny ("Zły")
# Traktujemy dane jako równomiernie próbkowane (ignorujemy luki czasu), czyli "ściskamy" szereg
f_naiwne, p_naiwne = compute_naive_periodogram(data_missing)

# 3. Periodogram Lomba-Scargle'a ("Dobry")
# Emulacja parametrów z R (pakiet 'lomb', funkcja lsp): ofac=4 lub 5
ofac = 4 
# Zakres częstotliwości: od bardzo małej do Nyquista (0.5)
# Zagęszczona siatka częstotliwości (oversampling)
total_duration = t_missing.max() - t_missing.min()
freqs = np.linspace(1/(total_duration*ofac), 0.5, int(total_duration * ofac * 0.5))
ang_freqs = 2 * np.pi * freqs

# Obliczamy Lomb-Scargle (znormalizowany)
# p_lomb = signal.lombscargle(t_missing, data_missing, ang_freqs, normalize=True)
p_lomb = signal.lombscargle(t_missing, data_missing, ang_freqs, normalize=False)

# Poziom istotności (False Alarm Probability) wg wzoru Baluev'a (używanego często w R/astro)
# Poniżej wzór klasyczny dla normalizacji 'standard' (Scargle 1982)
# z = -ln(1 - (1 - alpha)^(1/M))
M = 2 * len(freqs) / ofac # efektywna liczba niezależnych częstotliwości
alpha = 0.05
z_threshold = -np.log(1 - (1 - alpha)**(1/M))

# --- WIZUALIZACJA ---
# Rysujemy na OSOBNYCH subplotach, aby uniknąć problemów ze skalą
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=("1. Podejście błędne: Periodogram Naiwny (ignorowanie braków)", 
                                    "2. Podejście poprawne: Lomb-Scargle (uwzględnienie czasu)"),
                    vertical_spacing=0.15)

# Wykres 1: Naiwny
fig.add_trace(go.Scatter(x=f_naiwne[1:], y=p_naiwne[1:], 
                         name="Naiwny (niepoprawny)", line=dict(color='red', width=1)),
              row=1, col=1)

# Wykres 2: Lomb-Scargle + Próg
fig.add_trace(go.Scatter(x=freqs, y=p_lomb, 
                         name="Lomb-Scargle", line=dict(color='blue', width=1.5)),
              row=2, col=1)

fig.add_hline(y=z_threshold, line_dash="dash", line_color="green",
              annotation_text="Próg istotności 5%", annotation_position="top right",
              row=2, col=1)

fig.update_layout(
    title="Wpływ braków danych na analizę spektralną",
    height=800,
    showlegend=False
)

fig.update_xaxes(title_text="Częstotliwość", row=2, col=1)
fig.update_yaxes(title_text="Moc", row=1, col=1)
fig.update_yaxes(title_text="Moc Znormalizowana (PN)", row=2, col=1)

show_plotly(fig)

### Wnioski i Interpretacja (Punkt 3):

1. **Błąd metody naiwnej**: Traktowanie danych niekompletnych jako ciągłych powoduje przesunięcie częstotliwości i pojawienie się artefaktów (aliasing). Piki nie odpowiadają rzeczywistym cyklom.
2. **Przewaga Lomba-Scargle'a**: Metoda ta poprawnie uwzględnia czasy wystąpienia obserwacji, dzięki czemu piki częstotliwości pozostają na swoich miejscach mimo braku 30% danych.
3. **Istotność statystyczna**: Linia progu istotności pozwala odróżnić rzeczywiste sygnały od szumu wynikającego z braków danych. Piki wystające ponad zieloną linię mogą być uznane za wiarygodne.